In [ ]:
#data
import xarray as xr
import numpy as np 
import zarr
#plotting
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]
# gui elements
import ipywidgets as widgets
from ipyleaflet import *

In [ ]:
# History file form delft3d-fm run by Avelon's ZUNO-RMM model
# The data has been converted from netcdf to zarr format
# 
# The data is stored in Nextcloud with sharepath (almost regular http). Here at hetzner.de as a provider
his=xr.open_zarr("https://nx7384.your-storageshare.de/apps/sharingpath/wetwin/public/DCSM-FM_0_5nm_0000_his.zarr",consolidated=True)

# get locations and names
xs=his.station_x_coordinate.values
ys=his.station_y_coordinate.values
names=his.station_name.values.astype(str)

In [ ]:
# create output widget for a plot
out_plot = widgets.Output(layout = widgets.Layout(width="100%",height='300px'))
 
def update_timeseries_plot(locid):
    with out_plot:
        #plt.plot(his.time[:],his.waterlevel[:,locid])
        ifirst=5000
        ilast=7000 #max 315649
        plt.plot(his.time[ifirst:ilast],his.waterlevel[ifirst:ilast,locid])
        station=his.station_name[locid].values.astype(str)
        plt.title(station)
        out_plot.clear_output()
        plt.show()


In [ ]:
# background map zoomed at the RMM region
m = Map(center=(55.5, 1.5), zoom=4, basemap=basemaps.OpenStreetMap.Mapnik)
# 
from functools import partial

def update_location(event,type,coordinates,locid=0):
    update_timeseries_plot(locid)

# add stations as markers
station_markers=[]
for i in range(len(xs)):
    #marker=Marker(location=(ys[i], xs[i]),title=names[i])
    marker=CircleMarker(location=(ys[i], xs[i]),radius=3)
    f=partial(update_location,locid=i)
    marker.on_click(f)
    station_markers.append(marker)
station_cluster = MarkerCluster(markers=station_markers,max_cluster_radius=3)
m.add_layer(station_cluster);


In [ ]:
# initial plot
update_timeseries_plot(1)

In [ ]:
#  Build app layout

# big buttons for debugging
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style, 
        layout=widgets.Layout(height='auto', width='auto'))

#top_left_button = create_expanded_button("Top left", 'info')
#top_right_button = create_expanded_button("Top right", 'success')
bottom_left_button = create_expanded_button("Bottom left", 'danger')
bottom_right_button = create_expanded_button("Bottom right", 'warning')

#app = widgets.TwoByTwoLayout(top_left=m, top_right=out_plot,
#                            bottom_left=bottom_left_button, bottom_right=bottom_right_button)
app = widgets.TwoByTwoLayout(top_left=m, top_right=out_plot)

In [ ]:
display(app)